In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [2]:
#Loading dataset
data = pd.read_csv('fer2013.csv')
pixels = data['pixels'].tolist()

# Pre-processing: each pixels array is processed into a numpy array and added to the set
X = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(48, 48)
    face = np.expand_dims(face, axis=-1)
    X.append(face)

X = np.asarray(X)
X = X.astype('float32')
X /= 255.0  # Normlization [0,1]

# Getting the predicted labeles from the set
y = pd.get_dummies(data['emotion']).values  # One-hot encoding

# Divisione in train e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
#CNN model
#Sequential() --> stack of linear layer with one input and one output
model = tf.keras.models.Sequential([
    #Conv2D() conv levev with (num of filters, dim of filters, activate function, input shape --> 48X48 gray scale)
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    #MaxPooling2D() reducing the spacial dimension of the output (info preserved)
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    # Flatten() the 2D matrix of pixel is flattened into a 1D vector
    tf.keras.layers.Flatten(),
    #Dense(n, act) level denselt connected with n neuron with activation function act
    tf.keras.layers.Dense(256, activation='relu'),
    #Dropout(perc) used to prevent overfitting: it turn off casually perc neurons
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(7, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\acaia\anaconda3\envs\first_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
#Fitting the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 46s 98ms/step - accuracy: 0.2533 - loss: 1.8156 - val_accuracy: 0.3865 - val_loss: 1.5825
Epoch 2/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 48s 107ms/step - accuracy: 0.3968 - loss: 1.5465 - val_accuracy: 0.4703 - val_loss: 1.3778
Epoch 3/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 47s 105ms/step - accuracy: 0.4740 - loss: 1.3818 - val_accuracy: 0.5057 - val_loss: 1.3030
Epoch 4/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 48s 107ms/step - accuracy: 0.5135 - loss: 1.2888 - val_accuracy: 0.5272 - val_loss: 1.2388
Epoch 5/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 48s 107ms/step - accuracy: 0.5412 - loss: 1.2100 - val_accuracy: 0.5403 - val_loss: 1.2159
Epoch 6/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 48s 106ms/step - accuracy: 0.5630 - loss: 1.1508 - val_accuracy: 0.5444 - val_loss: 1.1844
Epoch 7/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 46s 103ms/step - accuracy: 0.5884 - loss: 1.0931 - val_accuracy: 0.5552 - val_loss: 1.1724
Epoch 8/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 45s 101ms/step - accuracy: 0.6021 - loss: 1.

In [5]:
#Testing
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

#Metrics calculation
accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

225/225 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step
Accuracy: 0.5707718027305656
Precision: 0.5720488463908726
Recall: 0.5707718027305656
F1 Score: 0.5698696835824126


In [6]:
model.save('cnn_FER2013.h5')